In [1]:
import pandas as pd 
import numpy as np

In [2]:
basefile = pd.read_csv("data/2020_Claims_SingleSource_v1.4.csv",dtype=str)
newfile = pd.read_csv("data/2021_Claims_SingleSource_v1.3.csv",dtype=str)

basefile.columns = basefile.columns.str.replace(' ', '_')
newfile.columns = newfile.columns.str.replace(' ', '_')

### Count comparison

In [3]:
basefile.count()

Measure_ID           19434
DATA_ELEMENT_NAME    19434
CODING_SYSTEM        19434
CODE                 19434
MODIFIER               883
PLACE_OF_SERVICE       724
AGE                  19434
GENDER               19434
dtype: int64

In [4]:
newfile.count()

Measure_ID           21380
DATA_ELEMENT_NAME    21380
CODING_SYSTEM        21380
CODE                 21380
MODIFIER               370
PLACE_OF_SERVICE       355
AGE                  21380
GENDER               21380
dtype: int64

### Joining the two files 

In [33]:
key=newfile.columns.values.tolist()

In [36]:
key

['Measure_ID',
 'DATA_ELEMENT_NAME',
 'CODING_SYSTEM',
 'CODE',
 'MODIFIER',
 'PLACE_OF_SERVICE',
 'AGE']

In [37]:
joint=newfile.merge(basefile,on=key,how="outer",suffixes=["_new","_base"])

In [38]:
joint

,Measure_ID,DATA_ELEMENT_NAME,CODING_SYSTEM,CODE,MODIFIER,PLACE_OF_SERVICE,AGE,GENDER_new,GENDER_base
0,1,ENCOUNTER_CODE,C4,99202,NaN,NaN,18 - 75,"M, F","M, F"
1,1,ENCOUNTER_CODE,C4,99203,NaN,NaN,18 - 75,"M, F","M, F"
2,1,ENCOUNTER_CODE,C4,99204,NaN,NaN,18 - 75,"M, F","M, F"
3,1,ENCOUNTER_CODE,C4,99205,NaN,NaN,18 - 75,"M, F","M, F"
4,1,ENCOUNTER_CODE,C4,99212,NaN,NaN,18 - 75,"M, F","M, F"
...,...,...,...,...,...,...,...,...,...
25968,437,G_CODE_PD,HCPCS,G9641,NaN,NaN,≥0,NaN,"M, F"
25969,437,G_CODE_PN,HCPCS,G9641,NaN,NaN,≥0,NaN,"M, F"
25970,437,G_CODE_PD_Exl,HCPCS,G9640,NaN,NaN,≥0,NaN,"M, F"
25971,437,G_CODE_PD,HCPCS,G9639,NaN,NaN,≥0,NaN,"M, F"


In [39]:
onlybase=joint[joint.GENDER_new.isna()]
len(onlybase)

4560

In [40]:
onlynew=joint[joint.GENDER_base.isna()]
len(onlynew)

6513

# New against Base comparison

## Measure ID analysis

In [43]:
newids=onlynew.Measure_ID.value_counts()
baseids=basefile.Measure_ID.value_counts()

#### MeasureIds introduced as part of the new version

In [63]:
newids.to_frame().join(baseids.to_frame(),how="left",lsuffix="_new",rsuffix="_base").head()

,Measure_ID_new,Measure_ID_base
236,781,55
418,590,3543
117,579,327
1,569,320
24,564,3479


## CODE Analysis

#### Codes introduced / moved as part of the new version

In [56]:
newden=onlynew.CODE.value_counts().convert_dtypes().to_frame()
baseden=basefile.CODE.value_counts().convert_dtypes().to_frame()

In [59]:
newcodes=newden.join(baseden,how="left",lsuffix="_new",rsuffix="_base")

#### Codes which are completely new

In [60]:
newcodes[newcodes["CODE_base"].isna()]

,CODE_new,CODE_base
L89.119,11,<NA>
L89.309,11,<NA>
L89.209,11,<NA>
L89.159,11,<NA>
L89.149,11,<NA>
...,...,...
90846,1,<NA>
90847,1,<NA>
G2181,1,<NA>
92507,1,<NA>


##### Moved codes

In [62]:
newcodes[~newcodes["CODE_base"].isna()]

,CODE_new,CODE_base
99348,9,20
99342,9,20
99350,9,20
99349,9,20
99344,9,20
...,...,...
90965,1,1
90966,1,1
90967,1,1
90968,1,1
